In [2]:
!pip install gradio pandas plotly tqdm numpy matplotlib

import gradio as gr
import pandas as pd
import random
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm import tqdm

import matplotlib
matplotlib.use('Agg')  # So we can render in memory
import matplotlib.pyplot as plt
import io
from PIL import Image

In [3]:
# ====================
# Data Classes
# ====================

class Order:
    """
    Represents a production order.
    
    The constructor extracts and stores key details from a data row.
    """
    def __init__(self, row):
        self.order_id = row['order_id']               # Unique identifier for the order
        self.product_type = row['Product type']         # Product type, used to determine if a setup time is needed
        self.cut_time = row['cut time']                 # Time required for the cutting stage
        self.sew_time = row['sew time']                 # Time required for the sewing stage
        self.pack_time = row['pack time']               # Time required for the packing stage
        self.deadline = row['deadline']                 # Delivery deadline for the order
        self.requires_delay = row['requires_out_of_factory_delay']  # Flag indicating if an additional delay is required

class Machine:
    """
    Represents a machine used in the production process.
    
    This class keeps track of the machine's unique identifier, its availability,
    the last product processed (to determine if setup time is needed), and its utilization history.
    """
    def __init__(self, machine_type, machine_id):
        self.machine_id = f"{machine_type}-{machine_id}"  # Unique identifier (e.g., "Cut-1")
        self.available_at = 0                             # The time at which the machine becomes available
        self.last_product = None                          # The last product processed on this machine
        self.utilization = []                             # List to record start and end times of tasks

def set_seed(seed_value=42):
    """
    Sets the seed for random number generation to ensure reproducible results.
    
    This function sets the seed for both Python's random module and numpy's random module.
    """
    random.seed(seed_value)  # Set seed for the random module
    np.random.seed(seed_value)  # Set seed for numpy's random module

def load_data(df):
    """
    Processes the DataFrame to convert the 'requires_out_of_factory_delay' column into boolean values.
    
    Conversion steps:
    1. Convert the column to a string.
    2. Strip any surrounding whitespace.
    3. Convert the string to uppercase.
    4. Map 'TRUE' to True and 'FALSE' to False.
    
    Returns:
        The processed DataFrame.
    """
    df['requires_out_of_factory_delay'] = (
        df['requires_out_of_factory_delay']
        .astype(str)
        .str.strip()
        .str.upper()
        .map({'TRUE': True, 'FALSE': False})
    )
    return df


In [4]:
# ====================
# Core Logic
# ====================

def simulate_schedule(orders, num_cutting, num_sewing, num_packing):
    """
    Schedules the production orders through the cutting, sewing, and packing stages.
    
    Parameters:
        orders (list): A list of Order objects.
        num_cutting (int): Number of available cutting machines.
        num_sewing (int): Number of available sewing machines.
        num_packing (int): Number of available packing machines.
        
    Returns:
        schedule (dict): Mapping of each order ID to its scheduled times and assigned machines.
        lateness_summary (dict): Mapping of each order ID to its lateness.
        total_lateness (int): Total lateness across all orders.
        on_time_count (int): Number of orders completed on time.
    """
    # Create machine instances for each stage based on provided numbers
    cutting_machines = [Machine('Cut', i+1) for i in range(num_cutting)]
    sewing_machines = [Machine('Sew', i+1) for i in range(num_sewing)]
    packing_machines = [Machine('Pack', i+1) for i in range(num_packing)]

    # Initialize dictionaries and counters for tracking the schedule and lateness
    schedule = {}
    lateness_summary = {}
    total_lateness = 0
    on_time_count = 0

    # Process each order sequentially
    for order in orders:
        # --------------------
        # Cutting Stage
        # --------------------
        # Calculate potential start times for the cutting stage on each machine,
        # considering a 10-unit setup time if the product type changes.
        cut_options = []
        for machine in cutting_machines:
            setup = 10 if (machine.last_product and machine.last_product != order.product_type) else 0
            start_time = machine.available_at + setup
            cut_options.append((start_time, machine))
            
        # Select the cutting machine with the earliest start time
        start_cut, best_cut = min(cut_options, key=lambda x: x[0])
        end_cut = start_cut + order.cut_time  # Compute finishing time for cutting stage
        best_cut.available_at = end_cut       # Update machine's availability
        best_cut.last_product = order.product_type  # Record the product type processed

        # --------------------
        # Sewing Stage
        # --------------------
        # Calculate when sewing can start; add a 48-unit delay if required
        sew_ready = end_cut + (48 if order.requires_delay else 0)
        # Determine the earliest available sewing machine after considering the delay
        sew_options = [(max(m.available_at, sew_ready), m) for m in sewing_machines]
        start_sew, best_sew = min(sew_options, key=lambda x: x[0])
        end_sew = start_sew + order.sew_time  # Compute finishing time for sewing stage
        best_sew.available_at = end_sew        # Update machine's availability

        # --------------------
        # Packing Stage
        # --------------------
        # Determine the earliest available packing machine after sewing is complete
        pack_options = [(max(m.available_at, end_sew), m) for m in packing_machines]
        start_pack, best_pack = min(pack_options, key=lambda x: x[0])
        end_pack = start_pack + order.pack_time  # Compute finishing time for packing stage
        
        # Conflict check: Ensure that the selected machine is truly available
        if start_pack < best_pack.available_at:
            raise ValueError("Packing conflict")
            
        best_pack.available_at = end_pack  # Update machine's availability

        # --------------------
        # Lateness Calculation
        # --------------------
        # Calculate lateness as the positive difference between finishing time and the deadline
        lateness = max(0, end_pack - order.deadline)
        total_lateness += lateness
        # Count order as on-time if there is no lateness
        on_time_count += 1 if lateness == 0 else 0
        
        # Record the scheduling details for this order
        schedule[order.order_id] = {
            'Cut': (best_cut.machine_id, start_cut, end_cut),
            'Sew': (best_sew.machine_id, start_sew, end_sew),
            'Pack': (best_pack.machine_id, start_pack, end_pack),
            'Lateness': lateness
        }
        lateness_summary[order.order_id] = lateness

    return schedule, lateness_summary, total_lateness, on_time_count

def generate_permutation(orders):
    """
    Generates a new permutation of orders based on one of three heuristics:
      - Deadline (60% chance)
      - Product type (20% chance)
      - Processing time (20% chance)
      
    Returns:
        shuffled (list): The permuted list of orders.
        heuristic (str): The name of the heuristic used.
    """
    r = random.random()
    if r < 0.6:  # Deadline heuristic (60%)
        sorted_orders = sorted(orders, key=lambda x: x.deadline)
        groups = {}
        # Group orders by deadline
        for order in sorted_orders:
            key = order.deadline
            if key not in groups:
                groups[key] = []
            groups[key].append(order)
        shuffled = []
        # Shuffle orders within each deadline group and combine
        for key in sorted(groups.keys()):
            random.shuffle(groups[key])
            shuffled.extend(groups[key])
        return shuffled, 'deadline'
    
    elif r < 0.8:  # Product type heuristic (20%)
        sorted_orders = sorted(orders, key=lambda x: x.product_type)
        groups = {}
        # Group orders by product type
        for order in sorted_orders:
            key = order.product_type
            if key not in groups:
                groups[key] = []
            groups[key].append(order)
        shuffled = []
        # Shuffle orders within each product type group and combine
        for key in sorted(groups.keys()):
            random.shuffle(groups[key])
            shuffled.extend(groups[key])
        return shuffled, 'product'
    
    else:  # Processing time heuristic (20%)
        sorted_orders = sorted(orders, key=lambda x: (x.cut_time + x.sew_time + x.pack_time))
        groups = {}
        # Group orders by total processing time
        for order in sorted_orders:
            key = order.cut_time + order.sew_time + order.pack_time
            if key not in groups:
                groups[key] = []
            groups[key].append(order)
        shuffled = []
        # Shuffle orders within each group and combine
        for key in sorted(groups.keys()):
            random.shuffle(groups[key])
            shuffled.extend(groups[key])
        return shuffled, 'processing_time'

def optimize_schedules(data, iterations, num_cutting, num_sewing, num_packing):
    """
    Uses a Monte Carlo simulation to optimize the production schedule.
    
    Steps:
      1. Set a fixed seed for reproducibility.
      2. Convert the DataFrame to a list of Order objects.
      3. For a given number of iterations:
         - Generate a permutation of orders using generate_permutation.
         - Run the simulation using simulate_schedule.
         - If a conflict occurs (e.g., packing conflict), skip the permutation.
         - Update the best schedule if the current simulation yields more on-time orders 
           or, when tied, lower total lateness.
         - Record performance metrics for visualization.
         
    Returns:
        best (dict): The best schedule and its performance metrics.
        on_time_hist (list): History of on-time order counts over iterations.
        lateness_hist (list): History of total lateness over iterations.
        heuristic_hist (list): History of heuristics used across iterations.
    """
    set_seed(42)  # Ensure reproducibility
    orders = [Order(row) for _, row in data.iterrows()]  # Create Order objects from data
    # Initialize best schedule with worst-case performance
    best = {'on_time': -1, 'lateness': float('inf'), 'schedule': None, 'lateness_map': {}}
    
    on_time_hist, lateness_hist, heuristic_hist = [], [], []

    # Iterate for the specified number of iterations
    for _ in tqdm(range(iterations)):
        # Generate a new permutation of orders using a random heuristic
        permuted, heuristic = generate_permutation(orders.copy())
        try:
            # Run the simulation on the permuted orders
            schedule, lateness_map, total_lat, on_time = simulate_schedule(
                permuted, num_cutting, num_sewing, num_packing
            )
        except ValueError:
            # Skip this iteration if a packing conflict occurs
            continue

        # Update the best schedule if the current one is better
        if on_time > best['on_time'] or (on_time == best['on_time'] and total_lat < best['lateness']):
            best.update(on_time=on_time, lateness=total_lat, 
                        schedule=schedule, lateness_map=lateness_map)
            
        # Record performance metrics for this iteration
        on_time_hist.append(best['on_time'])
        lateness_hist.append(best['lateness'])
        heuristic_hist.append(heuristic)
    
    return best, on_time_hist, lateness_hist, heuristic_hist


In [5]:
# ====================
# Visualization
# ====================

def create_lateness_chart(lateness_map):
    """
    Creates a bar chart showing the lateness distribution across orders.
    
    Parameters:
        lateness_map (dict): A dictionary mapping each order ID to its lateness value.
        
    Returns:
        fig (plotly.graph_objs._figure.Figure): A Plotly bar chart figure.
    """
    # Convert the lateness_map dictionary to a DataFrame for easier plotting
    df = pd.DataFrame(lateness_map.items(), columns=['Order', 'Lateness'])
    # Sort the DataFrame by Order ID for consistent plotting
    df = df.sort_values('Order')

    # Create a bar chart using Plotly Express
    fig = px.bar(
        df, 
        x='Order', 
        y='Lateness', 
        color='Lateness',  # Color bars based on the lateness value
        color_continuous_scale='RdYlGn_r',  # Use a red-yellow-green reversed color scale
        title='Order Lateness Distribution'
    )
    
    # Update the layout of the figure for better readability
    fig.update_layout(
        xaxis_title="Order ID",
        yaxis_title="Lateness (units)",
        plot_bgcolor='rgba(240,240,240,0.9)',  # Set a light grey background
        height=400
    )
    # Force x-axis to show every integer value (useful if Order IDs are numeric)
    fig.update_xaxes(dtick=1)

    return fig

def create_progress_chart(on_time, lateness):
    """
    Creates a dual-axis line chart showing the optimization progress.
    
    Parameters:
        on_time (list): History of on-time order counts over iterations.
        lateness (list): History of total lateness over iterations.
        
    Returns:
        fig (plotly.graph_objs._figure.Figure): A Plotly figure with two y-axes.
    """
    # Create a figure with secondary y-axis support
    fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    # Add a line trace for on-time orders (primary y-axis)
    fig.add_trace(
        go.Scatter(
            y=on_time, 
            name="On-Time Orders",
            line=dict(width=2)
        ),
        secondary_y=False
    )
    
    # Add a line trace for total lateness (secondary y-axis)
    fig.add_trace(
        go.Scatter(
            y=lateness, 
            name="Total Lateness",
            line=dict(width=2, dash='dot')
        ),
        secondary_y=True
    )
    
    # Update the layout of the chart
    fig.update_layout(
        title='Optimization Progress',
        xaxis_title="Iterations",
        plot_bgcolor='rgba(255,255,255,0.9)',  # Set a white background with slight transparency
        height=400,
        legend=dict(orientation="h", yanchor="bottom", y=1.02)
    )
    return fig

def create_heuristic_chart(heuristic_history):
    """
    Creates a pie chart showing the distribution of heuristics used during the simulation.
    
    Parameters:
        heuristic_history (list): A list of heuristic names used in each simulation iteration.
        
    Returns:
        fig (plotly.graph_objs._figure.Figure): A Plotly pie chart figure.
    """
    # Count the occurrences of each heuristic in the history
    counts = pd.Series(heuristic_history).value_counts().to_dict()
    # Create a pie chart using Plotly Express
    fig = px.pie(
        names=list(counts.keys()), 
        values=list(counts.values()), 
        hole=0.4,  # Create a donut chart style
        title="Heuristic Distribution"
    )
    # Update layout settings for better display
    fig.update_layout(showlegend=False, height=400)
    fig.update_traces(textposition='inside', textinfo='percent+label')
    return fig

def create_machine_timeline_image(schedule):
    """
    Generates a Gantt-like chart (machine utilization timeline) using Matplotlib,
    converts the chart to a PIL image, and returns it.
    
    Parameters:
        schedule (dict): A dictionary mapping each order ID to its scheduled details (for Cut, Sew, Pack stages).
        
    Returns:
        PIL.Image: An in-memory PIL image of the machine utilization timeline.
    """
    # Group tasks by machine to prepare for plotting
    machines = {}
    for order_id, stages in schedule.items():
        for stage in ['Cut', 'Sew', 'Pack']:
            machine_id, start, end = stages[stage]
            if machine_id not in machines:
                machines[machine_id] = []
            machines[machine_id].append((order_id, start, end))
    
    # Create a figure and axis for the Gantt chart using Matplotlib
    fig, ax = plt.subplots(figsize=(10, 6))
    colors = plt.cm.tab20.colors  # Use a color map with many distinct colors

    # Plot each machine's tasks as horizontal bars
    for idx, (machine, tasks) in enumerate(machines.items()):
        for task in tasks:
            order_id, start, finish = task
            ax.barh(
                y=machine, 
                width=finish - start,  # Duration of the task
                left=start,            # Starting time of the task
                color=colors[idx % 20],  # Cycle through colors based on machine index
                edgecolor='black'
            )

    # Label the axes and set a title for the chart
    ax.set_xlabel('Time Units')
    ax.set_title('Machine Utilization Timeline')
    plt.tight_layout()

    # Save the figure to a BytesIO buffer and convert it to a PIL image
    buf = io.BytesIO()
    fig.savefig(buf, format='png', dpi=100)
    plt.close(fig)  # Close the figure to free memory
    buf.seek(0)
    return Image.open(buf)


In [6]:
# ====================
# Gradio Interface
# ====================

def run_simulation(file, cutting, sewing, packing, algorithm, iterations):
    # Set the random seed for reproducibility
    set_seed(42)
    
    # Load CSV data from the uploaded file and process it
    df = load_data(pd.read_csv(file.name))
    
    # Run the simulation using the provided parameters:
    # - Number of iterations (converted to int)
    # - Number of cutting, sewing, and packing machines (converted to int)
    best, on_time, lateness, heuristic = optimize_schedules(
        df, 
        int(iterations), 
        int(cutting), 
        int(sewing), 
        int(packing)
    )
    
    # Create a Gantt chart image to visualize machine utilization
    gantt_img = create_machine_timeline_image(best['schedule'])
    
    # Return a tuple containing:
    # - Formatted on-time order count
    # - Formatted average lateness per order
    # - Lateness distribution chart (Plotly figure)
    # - Optimization progress chart (Plotly figure)
    # - Heuristic distribution chart (Plotly figure)
    # - Gantt chart image (PIL image)
    return (
        f"**{best['on_time']}/{len(df)}**",  # On-Time Orders
        f"**{best['lateness']/len(df):.2f} units**",  # Average Lateness
        create_lateness_chart(best['lateness_map']),
        create_progress_chart(on_time, lateness),
        create_heuristic_chart(heuristic),
        gantt_img
    )

# Define the Gradio Blocks interface with a soft theme and custom CSS for styling numbers
with gr.Blocks(
    theme=gr.themes.Soft(),
    css=".numbers {font-size: 32px !important; text-align: center; margin: 15px 0;}"
) as app:
    # Add a Markdown header for the app title
    gr.Markdown("# 🏭 Production Scheduler")
    
    with gr.Row():
        # Left Column: Configuration panel
        with gr.Column():
            gr.Markdown("## ⚙️ Configuration")
            with gr.Row():
                # Number inputs for machine configurations
                cutting = gr.Number(value=2, label="Cutting Tables", minimum=1)
                sewing = gr.Number(value=3, label="Sewing Machines", minimum=1)
                packing = gr.Number(value=1, label="Packing Stations", minimum=1)
            # File upload widget for the orders CSV
            data = gr.File(label="Upload Orders CSV")
            # Slider to select the number of simulation iterations
            iterations = gr.Slider(100, 500, step=50, value=500, label="Simulation Iterations")
            
            # Dropdown to select the simulation algorithm (only "Monte Carlo" is available)
            algo_dropdown = gr.Dropdown(["Monte Carlo"], label="Algorithm", value="Monte Carlo")
            # Button to trigger the simulation
            run = gr.Button("🚀 Start Simulation", variant="primary")
            
        # Right Column: Results panel
        with gr.Column():
            gr.Markdown("## 📊 Results")
            with gr.Row():
                with gr.Column():
                    # Display for on-time orders
                    gr.Markdown("✅ On-Time Orders")
                    on_time_output = gr.Markdown(elem_classes="numbers")
                with gr.Column():
                    # Display for average lateness
                    gr.Markdown("⏱️ Average Lateness")
                    avg_late_output = gr.Markdown(elem_classes="numbers")
            
            # Tabs for displaying different charts
            with gr.Tabs():
                with gr.Tab("📈 Lateness "):
                    # Tab for lateness distribution chart
                    lateness_chart = gr.Plot()
                with gr.Tab("📉 Optimization "):
                    # Tab for optimization progress chart
                    progress_chart = gr.Plot()
                with gr.Tab("📊 Strategy Analysis"):
                    # Tab for heuristic analysis chart
                    heuristic_chart = gr.Plot()
                with gr.Tab("🛠 Machine Utilization"):
                    # Tab for the machine utilization (Gantt chart) image
                    gantt_chart = gr.Image(type="pil")

    # Bind the "Start Simulation" button click event:
    # When clicked, the run_simulation function is called with the user inputs,
    # and its outputs are directed to the corresponding UI components.
    run.click(
        run_simulation,
        inputs=[data, cutting, sewing, packing, algo_dropdown, iterations],
        outputs=[
            on_time_output, 
            avg_late_output, 
            lateness_chart, 
            progress_chart, 
            heuristic_chart,
            gantt_chart
        ]
    )

# Launch the Gradio app with options for inline display and public sharing
if __name__ == "__main__":
    app.launch(inline=True, share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://8c8747c5d5073f31d6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
